# Airbnb data investigation
## Seattle Airbnb Open Data
> A sneak peek into the Airbnb activity in Seattle, WA, USA
### Context
> Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Seattle, WA.

### Content
> The following Airbnb activity is included in this Seattle dataset:

>* Listings, including full descriptions and average review score
>* Reviews, including unique id for each reviewer and detailed comments
>* Calendar, including listing id and the price and availability for that day

### Acknowledgement
> This dataset is part of Airbnb Inside, and the original source can be found [here](http://insideairbnb.com/get-the-data.html).

## Boston Airbnb Open Data
> A sneak peek into the Airbnb activity in Boston, MA, USA
### Context
> Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Boston, MA.

### Content
> The following Airbnb activity is included in this Boston dataset:

> *Listings, including full descriptions and average review score
> *Reviews, including unique id for each reviewer and detailed comments
> *Calendar, including listing id and the price and availability for that day

### Acknowledgement
> This dataset is part of Airbnb Inside, and the original source can be found [here](http://insideairbnb.com/get-the-data.html).

# Load data

In [7]:
# data location
%ls ../../Datasets

Boston Airbnb Open Data/        Dataset of USED CARS.zip
Boston Airbnb Open Data.zip     Netflix_movie_and_TV_shows.csv
Car Sales.xlsx - car_data.csv   Netflix_movie_and_TV_shows.zip
Car sales report.zip            Seattle_Airbnb/
Dataset of USED CARS.csv        Seattle_Airbnb.zip


In [60]:
# set data location
data_dir = '../../Datasets/'
boston_dir = data_dir+"Boston Airbnb Open Data/"
seattle_dir = data_dir+'Seattle_Airbnb/'

In [75]:
import os
# all boston datasets and seattle datasets
bs_all,sa_all = [],[]
for root,dirs,files in os.walk(boston_dir):
    for file in files:
        bs_all.append(os.path.join(root,file))
for root,dirs,files in os.walk(seattle_dir):
    for file in files:
        sa_all.append(os.path.join(root,file))

In [76]:
bs_all

['../../Datasets/Boston Airbnb Open Data/reviews.csv',
 '../../Datasets/Boston Airbnb Open Data/listings.csv',
 '../../Datasets/Boston Airbnb Open Data/calendar.csv']

In [77]:
sa_all

['../../Datasets/Seattle_Airbnb/reviews.csv',
 '../../Datasets/Seattle_Airbnb/listings.csv',
 '../../Datasets/Seattle_Airbnb/calendar.csv']

> ## Load all datasets